In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests

# Import the API key.
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Sabang,ID,5.8933,95.3214,82.47,74,46,11.18,scattered clouds
1,1,Yellowknife,CA,62.4560,-114.3525,14.29,63,100,5.99,overcast clouds
2,2,Iqaluit,CA,63.7506,-68.5145,-5.31,70,18,15.99,few clouds
3,3,Ushuaia,AR,-54.8000,-68.3000,45.88,77,100,3.85,light rain
4,4,Saint-Pierre,RE,-21.3393,55.4781,78.48,91,0,4.61,clear sky


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your vacation? "))
max_temp = float(input("What is the maximum temperature you would like for your vacation? "))

What is the minimum temperature you would like for your vacation? 70
What is the maximum temperature you would like for your vacation? 93


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Sabang,ID,5.8933,95.3214,82.47,74,46,11.18,scattered clouds
4,4,Saint-Pierre,RE,-21.3393,55.4781,78.48,91,0,4.61,clear sky
5,5,Baturaja,ID,-4.1333,104.1667,72.14,94,92,2.15,overcast clouds
8,8,Hermanus,ZA,-34.4187,19.2345,71.80,82,84,15.01,broken clouds
11,11,Hilo,US,19.7297,-155.0900,80.33,72,75,4.61,light rain
13,13,Rikitea,PF,-23.1203,-134.9692,79.77,71,63,12.39,broken clouds
19,19,Ngaoundere,CM,7.3167,13.5833,75.16,17,94,3.40,overcast clouds
20,20,Souillac,MU,-20.5167,57.5167,80.06,87,23,10.87,few clouds
22,22,Kavieng,PG,-2.5744,150.7967,81.37,77,100,8.79,light rain
24,24,Dingle,PH,10.9995,122.6711,76.91,91,100,6.02,overcast clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()


City_ID                296
City                   296
Country                296
Lat                    296
Lng                    296
Max Temp               296
Humidity               296
Cloudiness             296
Wind Speed             296
Current Description    296
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
# no need, there are no empty rows

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Sabang,ID,82.47,scattered clouds,5.8933,95.3214,
4,Saint-Pierre,RE,78.48,clear sky,-21.3393,55.4781,
5,Baturaja,ID,72.14,overcast clouds,-4.1333,104.1667,
8,Hermanus,ZA,71.80,broken clouds,-34.4187,19.2345,
11,Hilo,US,80.33,light rain,19.7297,-155.0900,
13,Rikitea,PF,79.77,broken clouds,-23.1203,-134.9692,
19,Ngaoundere,CM,75.16,overcast clouds,7.3167,13.5833,
20,Souillac,MU,80.06,few clouds,-20.5167,57.5167,
22,Kavieng,PG,81.37,light rain,-2.5744,150.7967,
24,Dingle,PH,76.91,overcast clouds,10.9995,122.6711,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Sabang,ID,82.47,scattered clouds,5.8933,95.3214,Hotel Citra
4,Saint-Pierre,RE,78.48,clear sky,-21.3393,55.4781,Lindsey Hôtel
5,Baturaja,ID,72.14,overcast clouds,-4.1333,104.1667,"HOTEL Sekar Sunja, Baturaja Timur, Ogan Komeri..."
8,Hermanus,ZA,71.80,broken clouds,-34.4187,19.2345,Misty Waves Boutique Hotel
11,Hilo,US,80.33,light rain,19.7297,-155.0900,Hilo Hawaiian Hotel
13,Rikitea,PF,79.77,broken clouds,-23.1203,-134.9692,People ThankYou
19,Ngaoundere,CM,75.16,overcast clouds,7.3167,13.5833,Hotel Mentong Palace
20,Souillac,MU,80.06,few clouds,-20.5167,57.5167,Shanti Maurice Resort & Spa
22,Kavieng,PG,81.37,light rain,-2.5744,150.7967,Nusa Island Retreat
24,Dingle,PH,76.91,overcast clouds,10.9995,122.6711,SEGAYA PROPERTY


In [10]:
# 7. Drop the rows where there is no Hotel Name.
#Based on the count there are no empty rows. But if there were, I would've used hotel_df.dropna(how ="any")

hotel_df.count()

City                   296
Country                296
Max Temp               296
Current Description    296
Lat                    296
Lng                    296
Hotel Name             296
dtype: int64

In [11]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [14]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [15]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))